In [ ]:
#default_exp approach

In [1]:
#hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Approach


> This module contains all the code for defining the various approaches

In [2]:
#export
import copy
import cv2
import multiprocessing
import pickle

import numpy as np

from collections import defaultdict, OrderedDict
from itertools import combinations_with_replacement
from joblib import Parallel, delayed
from pathlib import Path

# tango
from tango.eval import *
from tango.features import *
from tango.model import *
from tango.prep import *

from tqdm.auto import tqdm

from sklearn.metrics.pairwise import cosine_similarity

In [3]:
def gen_all_codebooks(imgs, models, vw):
    for i, model in enumerate(models):
        codebook = gen_vcodebook(imgs, model, vw)
        fname = f'/tf/data/models/cookbook_M{i:02}_{len(imgs)}n_{vw}vw.model'
        pickle.dump(codebook, open(fname, 'wb'))

In [4]:
# def gen_tfidfs(vid_ds, mdl, vw, codebook, df, ftk):
#     vid_tfids = defaultdict(
#         lambda: defaultdict(list)
#     )
#     for app, reports in tqdm(vid_ds.labels.items()):
# #         if app != 'car_report': continue
#         print(app)
#         for i, (report, vids) in enumerate(reports.items()):
#             for vid in vids:
#                 bovw = new_get_bovw(vid, mdl, codebook, vw, frames_to_keep = ftk)
#                 vid_tfids[app][report].append(calc_tf_idf(bovw, df))
    
#     return vid_tfids

In [47]:
def gen_tfidfs(vid_ds_features, vw, codebook, df, ftk):
    vid_tfids = defaultdict(
        lambda: defaultdict(list)
    )
    
    for app in vid_ds_features:
        for report in vid_ds_features[app]:
            for vid in vid_ds_features[app][report]:
                bovw = new_get_bovw(
                    vid_ds_features[app][report][vid]['features'],
                    codebook, vw
                )
                vid_tfids[app][report].append(calc_tf_idf(bovw, df))
    
#     for app, reports in tqdm(vid_ds.labels.items()):
# #         if app != 'car_report': continue
#         print(app)
#         for i, (report, vids) in enumerate(reports.items()):
#             for vid in vids:
# #                 bovw = new_get_bovw(vid, mdl, codebook, vw, frames_to_keep = ftk)
#                 bovw = new_get_bovw(features, codebook, vw, frames_to_keep = ftk)
#                 vid_tfids[app][report].append(calc_tf_idf(bovw, df))
    
    return vid_tfids

In [28]:
def gen_extracted_features(vid_ds, mdl, fps, ftk):
    vid_ds_features = {}
    for app in tqdm(vid_ds.labels):
        vid_ds_features[app] = {}
        for report in vid_ds[app]:
            vid_ds_features[app][report] = {}
            for i in range(len(vid_ds[app][report])):
                vid_ds_features[app][report][f'vid_{i}'] = {
                    'features': extract_features(vid_ds[app][report][i], mdl, fps, frames_to_keep = ftk)
                }
    
    return vid_ds_features

In [6]:
# def gen_similarity(vid_ds, mdl, codebook, vw, ftk):
#     results = {}
    
#     df = np.histogram(codebook.labels_, bins = range(vw + 1))[0]
#     vid_tfids = gen_tfidfs(vid_ds, mdl, vw, codebook, df, ftk)
#     for app, reports in vid_ds.labels.items():
#         if app != 'car_report': continue
#         results[app] = {}
#         for report_i in reports:
#             results[app][report_i] = {}
#             for report_j in reports:
#                 results[app][report_i][report_j] = {}
#                 for k in range(len(vid_tfids[app][report_i])):
#                     results[app][report_i][report_j][f'vid_{k}'] = {}
#                     for l in range(len(vid_tfids[app][report_j])):
#                         results[app][report_i][report_j][f'vid_{k}'][f'vid_{l}'] = np.dot(vid_tfids[app][report_i][k], vid_tfids[app][report_j][l]) / (np.linalg.norm(vid_tfids[app][report_i][k]) * np.linalg.norm(vid_tfids[app][report_j][l]))
    
#     return results

In [33]:
def gen_similarity(vid_ds, vid_ds_features, mdl, codebook, vw, ftk):
    results = defaultdict(
        lambda: defaultdict(
            lambda: defaultdict(
                lambda: defaultdict(
                    lambda: defaultdict(int)
                )
            )
        )
    )
    
    df = np.histogram(codebook.labels_, bins = range(vw + 1))[0]
    vid_tfids = gen_tfidfs(vid_ds_features, vw, codebook, df, ftk)
    for app, reports in vid_ds.labels.items():
#         if app != 'car_report': continue
        l = [(report, i) for report in reports for i in range(len(reports[report]))]
        pairs = list(x for x in combinations_with_replacement(l, 2) if x[0] != x[1])
        for (report_i, i), (report_j, j) in pairs:
            results[app][report_i][f'vid_{i}'][report_j][f'vid_{j}'] = np.dot(vid_tfids[app][report_i][i], vid_tfids[app][report_j][j]) / (np.linalg.norm(vid_tfids[app][report_i][i]) * np.linalg.norm(vid_tfids[app][report_j][j]))

    return results

In [34]:
def filter_vids(vid_similarities, mdl_vid_threshold = 0.8):
    vid_dict = copy.deepcopy(vid_similarities)
    for app in vid_dict:
        for report_i in vid_dict[app]:
            for report_j in vid_dict[app][report_i]:
                for vid_i in vid_dict[app][report_i][report_j]:
                    for vid_j in vid_dict[app][report_i][report_j][vid_i]:
                        if vid_dict[app][report_i][report_j][vid_i] < mdl_threshold:
                            del vid_dict[app][report_i][report_j][vid_i]
    
    return vid_dict

In [35]:
# def sift_frame_sim(mdl, codebook, frame_i, frame_j, vw, mdl_frame_threshold):
#     if frame_i is None or frame_j is None: return 0
#     features_i = mdl.extract(frame_i)
#     vws_i = np.expand_dims(codebook.predict(features_i), axis=0)
#     bowv_i = np.expand_dims(np.histogram(vws_i, bins = range(vw + 1))[0], axis=0)
    
#     features_j = mdl.extract(frame_j)
#     vws_j = np.expand_dims(codebook.predict(features_j), axis=0)
#     bowv_j = np.expand_dims(np.histogram(vws_j, bins = range(vw + 1))[0], axis=0)
    
#     sim = cosine_similarity(bowv_i, bowv_j)[0][0]
#     return sim

In [80]:
def sift_frame_sim(features_i, features_j, codebook, df, vw):
    vws_i = np.expand_dims(codebook.predict(features_i), axis=0)
    bowv_i = np.expand_dims(np.histogram(vws_i, bins = range(vw + 1))[0], axis=0)
#     tf_idf_i = calc_tf_idf(bowv_i, df)
    
    vws_j = np.expand_dims(codebook.predict(features_j), axis=0)
    bowv_j = np.expand_dims(np.histogram(vws_j, bins = range(vw + 1))[0], axis=0)
#     tf_idf_j = calc_tf_idf(bowv_j, df)
#     sim = np.dot(tf_idf_i, tf_idf_j) / (np.linalg.norm(tf_idf_i) * np.linalg.norm(tf_idf_j))
    # TODO Add getting tfidf and then performing cosine...
    sim = cosine_similarity(bowv_i, bowv_j)[0][0]
    return sim

In [37]:
# def simclr_frame_sim(mdl, codebook, frame_i, frame_j, vw, mdl_frame_threshold):
#     if frame_i is None or frame_j is None: return 0
#     features_i = mdl.extract(frame_i)
#     features_j = mdl.extract(frame_j)
    
#     sim = cosine_similarity(features_i, features_j)[0][0]
#     return sim

In [38]:
def simclr_frame_sim(features_i, features_j, codebook, vw):
    sim = cosine_similarity(features_i, features_j)[0][0]
    return sim

In [39]:
# # Modified from geeksforgeeks: https://www.geeksforgeeks.org/longest-common-substring-dp-29/
# def fuzzy_LCS(X, Y, m, n, sim_func, mdl, codebook, vw, mdl_frame_threshold, fps = 30, frames_to_keep = 1):
#     LCSuff = [[0 for k in range(n + 1)] for l in range(m + 1)] 
      
#     # To store the length of  
#     # longest common substring 
#     result = 0 
  
#     # Following steps to build 
#     # LCSuff[m+1][n+1] in bottom up fashion 
#     for i in range(0, m + 1):
#         for j in range(0, n + 1):
#             if (i == 0 or j == 0): 
#                 LCSuff[i][j] = 0
#                 continue
#             sim = sim_func(mdl, codebook, X[i * int(fps / frames_to_keep) -1], Y[j * int(fps / frames_to_keep) -1], vw, mdl_frame_threshold)
# #             print('SIM:', sim)
#             if sim > mdl_frame_threshold: 
#                 LCSuff[i][j] = LCSuff[i-1][j-1] + sim
#                 result = max(result, LCSuff[i][j]) 
#             else: 
#                 LCSuff[i][j] = 0
# #     print('Fuzzy:', result, min(m, n))
#     return result / min(m, n)

In [81]:
# Modified from geeksforgeeks: https://www.geeksforgeeks.org/longest-common-substring-dp-29/
def fuzzy_LCS(X, Y, m, n, sim_func, codebook, vw, mdl_frame_threshold = 0.0):
    LCSuff = [[0 for k in range(n + 1)] for l in range(m + 1)] 
      
    # To store the length of  
    # longest common substring 
    result = 0 
  
    # Following steps to build 
    # LCSuff[m+1][n+1] in bottom up fashion 
    for i in range(0, m + 1):
        for j in range(0, n + 1):
            if (i == 0 or j == 0): 
                LCSuff[i][j] = 0
                continue
            sim = sim_func(X[i - 1], Y[j - 1], codebook, vw)
#             print('SIM:', sim)
            if sim > mdl_frame_threshold:
                LCSuff[i][j] = LCSuff[i-1][j-1] + sim
                result = max(result, LCSuff[i][j]) 
            else: 
                LCSuff[i][j] = 0
#     print('Fuzzy:', result, min(m, n))
    return result / min(m, n)

In [41]:
def flatten_dict(d_in, d_out, parent_key):
    for k, v in d_in.items():
        if isinstance(v, dict):
            flatten_dict(v, d_out, parent_key + (k,))
        else:
            d_out[parent_key + (k,)] = v

In [42]:
def fix_corpus(results, corpus_size):
    
    for app in results:
#         corpus_size = len(results[app])
        s0 = 0
        v0 = 1
        for i in range(corpus_size):
            if v0 % 4 == 0: s0 += 1
            s1 = 0
            for v1 in range(i + 1):
                if v1 % 4 == 0: s1 += 1
                results[app][f'S{s0}'][f'vid_{v0 % 4}'][f'S{s1 - 1}'][f'vid_{v1 % 4}'] = results[app][f'S{s1 - 1}'][f'vid_{v1 % 4}'][f'S{s0}'][f'vid_{v0 % 4}']
#                 results[app][f'S{s0}'][f'vid_{v0 % 4}'][(f'S{s1 - 1}', f'vid_{v1 % 4}')] = results[app][f'S{s1 - 1}'][f'vid_{v1 % 4}'][(f'S{s0}', f'vid_{v0 % 4}')]
                
#             print('-' * 5)
            v0 += 1
    return results
#     print(i + 1)

In [43]:
def sort_results(results):
    sorted_results = {}
    for app in results:
        sorted_results[app] = {}
        for report in results[app]:
            sorted_results[app][report] = {}
            for vid in results[app][report]:
                sorted_results[app][report][vid] = []
                d_out = {}
                flatten_dict(results[app][report][vid], d_out, tuple())
                sorted_results[app][report][vid] = OrderedDict(
                    sorted(d_out.items(), key=lambda x: str(x[1]), reverse = True)
                )
    
    return sorted_results

In [55]:
def approach(
    vid_ds, mdl, mdl_vid_threshold, sim_func,
    mdl_frame_threshold, codebook, vw, corpus_size,
    fps = 30, ftk = 1, align_sim = False
):
    vid_ds_sims = gen_similarity(vid_ds, mdl, codebook, vw, ftk)
    if align_sim:
        for app, reports in vid_ds.labels.items():
    #         if app != 'car_report': continue
            l = [(report, i) for report in reports for i in range(len(reports[report]))]
            pairs = list(x for x in combinations_with_replacement(l, 2) if x[0] != x[1])
            for (report_i, i), (report_j, j) in tqdm(pairs):
                tfidf_sim = vid_ds_sims[app][report_i][f'vid_{i}'][report_j][f'vid_{j}']
                lcs_sim = fuzzy_LCS(
                    vid_ds[app][report_i][i], vid_ds[app][report_j][j],
                    int(len(vid_ds[app][report_i][i]) / int(fps / ftk)),
                    int(len(vid_ds[app][report_j][j]) / int(fps / ftk)),
                    sim_func, mdl, codebook, vw, mdl_frame_threshold, fps, ftk
                )
                vid_ds_sims[app][report_i][f'vid_{i}'][report_j][f'vid_{j}'] = (tfidf_sim + lcs_sim) / 2
#     for app, reports in vid_ds.labels.items():
#         if app != 'car_report': continue
#         for i, report_i in tqdm(enumerate(reports), total = len(reports)):
# #             if i > 1: break
#             for j, report_j in tqdm(enumerate(reports), total = len(reports)):
# #                 if j > 1: break
#                 for k in range(len(vid_ds[app][report_i])):
# #                     if k > 1: break
#                     for l in range(len(vid_ds[app][report_j])):
# #                         if l > 1: break
#                         tfidf_sim = vid_ds_sims[app][report_i][report_j][f'vid_{k}'][f'vid_{l}']
#                         lcs_sim = fuzzy_LCS(
#                             vid_ds[app][report_i][k], vid_ds[app][report_j][l],
#                             int(len(vid_ds[app][report_i][k]) / int(fps / ftk)), int(len(vid_ds[app][report_i][l]) / int(fps / ftk)),
#                             sim_func, mdl, codebook, vw, mdl_frame_threshold, fps, ftk
#                         )
#                         print(tfidf_sim, lcs_sim)
#                         vid_ds_sims[app][report_i][report_j][f'vid_{k}'][f'vid_{l}'] = (tfidf_sim + lcs_sim) / 2
#     print(vid_ds_sims)

    vid_ds_sims = fix_corpus(vid_ds_sims, corpus_size)
    results = sort_results(vid_ds_sims)
    return results

In [100]:
def cached_approach(
    vid_ds, mdl, mdl_vid_threshold, sim_func,
    mdl_frame_threshold, codebook, vw, corpus_size,
    fps = 30, ftk = 1, align_sim = False
):
    vid_ds_features = gen_extracted_features(vid_ds, mdl, fps, ftk)
    vid_ds_sims = gen_similarity(vid_ds, vid_ds_features, mdl, codebook, vw, ftk)
    if align_sim:
        for app, reports in vid_ds.labels.items():
    #         if app != 'car_report': continue
            l = [(report, i) for report in reports for i in range(len(reports[report]))]
            pairs = list(x for x in combinations_with_replacement(l, 2) if x[0] != x[1])
            for (report_i, i), (report_j, j) in tqdm(pairs):
                tfidf_sim = vid_ds_sims[app][report_i][f'vid_{i}'][report_j][f'vid_{j}']
                lcs_sim = fuzzy_LCS(
                    vid_ds_features[app][report_i][f'vid_{i}']['features'],
                    vid_ds_features[app][report_j][f'vid_{j}']['features'],
                    int(len(vid_ds[app][report_i][i]) / int(fps / ftk)),
                    int(len(vid_ds[app][report_j][j]) / int(fps / ftk)),
                    # len(vid_ds_features[app][report_i][f'vid_{i}']['features']) - 1,
                    # len(vid_ds_features[app][report_j][f'vid_{j}']['features']) - 1,
                    sim_func, codebook, vw, mdl_frame_threshold
                )
                vid_ds_sims[app][report_i][f'vid_{i}'][report_j][f'vid_{j}'] = (tfidf_sim + lcs_sim) / 2

    vid_ds_sims = fix_corpus(vid_ds_sims, corpus_size)
    results = sort_results(vid_ds_sims)
    return results

In [20]:
def get_eval_results(evals, app, item):
    for s in evals[app]:
        for vid in evals[app][s]:
            try:
                print(evals[app][s][vid][item])
            except: continue

In [21]:
l = [(f'S{i}', f'vid{j}') for i in range(5) for j in range(4)]
pairs = list(x for x in combinations_with_replacement(l, 2) if x[0] != x[1])

In [22]:
path = Path("/tf/data/datasets/validation_set")
vid_ds = VideoDataset.from_path(path).label_from_paths()
vid_ds.get_labels()

['car_report', 'king', 'tasty']

In [26]:
mdl_00_vid_threshold = 0.9
mdl_00_frame_threshold = 0.9
vw = 100

M00 = SIFTExtractor(cv2.xfeatures2d.SIFT_create())
fname = f'/tf/data/models/cookbook_M00_1000n_100vw.model'
codebook_00 = pickle.load(open(fname, 'rb'))

In [ ]:
results_00 = approach(
    vid_ds, M00, mdl_00_vid_threshold, sift_frame_sim,
    mdl_00_frame_threshold, codebook_00, vw, 19, fps = 30, ftk = 1
)

In [ ]:
results_00_bovw_align = approach(
    vid_ds, M00, mdl_00_vid_threshold, sift_frame_sim,
    mdl_00_frame_threshold, codebook_00, vw, 19, fps = 30, ftk = 1,
    align_sim = True
)

In [35]:
evals_00 = evaluate(results_00) # ftk = 1

car_report σ Rank 2.277608394786075
car_report μ Rank 2.25
car_report Median Rank 1.5
car_report mRR: 0.6937121212121212
car_report mAP: 0.5326424678262913
king σ Rank 4.363484845854286
king μ Rank 3.4
king Median Rank 1.0
king mRR: 0.6980373303167421
king mAP: 0.4601591918716531
tasty σ Rank 0.7810249675906654
tasty μ Rank 1.3
tasty Median Rank 1.0
tasty mRR: 0.9041666666666666
tasty mAP: 0.8303373015873016


In [60]:
get_eval_results(evals_00, 'tasty', 'average_precision')

0.3222222222222222
0.29444444444444445
0.7916666666666666
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.7916666666666666
0.5555555555555555
0.8666666666666667
0.625
0.7666666666666666
0.8095238095238094
0.9166666666666666
0.8666666666666667


TypeError: 'numpy.float64' object is not iterable

In [69]:
results_00_cache = cached_approach(
    vid_ds, M00, mdl_00_vid_threshold, simclr_frame_sim,
    mdl_00_frame_threshold, codebook_00, vw, 19, fps = 30, ftk = 1
)

In [70]:
evaluate(results_00_cache) # ftk = 1

car_report σ Rank 2.277608394786075
car_report μ Rank 2.25
car_report Median Rank 1.5
car_report mRR: 0.6937121212121212
car_report mAP: 0.5326424678262913
car_report Hit@5: 0.95
king σ Rank 4.363484845854286
king μ Rank 3.4
king Median Rank 1.0
king mRR: 0.6980373303167421
king mAP: 0.4601591918716531
king Hit@5: 0.8
tasty σ Rank 0.7810249675906654
tasty μ Rank 1.3
tasty Median Rank 1.0
tasty mRR: 0.9041666666666666
tasty mAP: 0.8303373015873016
tasty Hit@5: 1.0


{'car_report': {'S0': {'vid_0': {'ranks': [(('S4', 'vid_0'),
      0.9954718971541773),
     (('S4', 'vid_2'), 0.9945964770493874),
     (('S3', 'vid_0'), 0.9939976838697654),
     (('S3', 'vid_2'), 0.9931883158114038),
     (('S0', 'vid_2'), 0.9929479446868809),
     (('S4', 'vid_3'), 0.9928277720624915),
     (('S2', 'vid_0'), 0.9924880112472121),
     (('S0', 'vid_3'), 0.9916407619640583),
     (('S3', 'vid_3'), 0.9916281614374629),
     (('S3', 'vid_1'), 0.9908729735712648),
     (('S0', 'vid_1'), 0.9904742624406746),
     (('S4', 'vid_1'), 0.9896292570686976),
     (('S1', 'vid_2'), 0.9869854517117607),
     (('S1', 'vid_0'), 0.9865547211253137),
     (('S1', 'vid_3'), 0.986118447341411),
     (('S2', 'vid_1'), 0.9850550846216436),
     (('S2', 'vid_3'), 0.983298468501515),
     (('S1', 'vid_1'), 0.9793310935304032),
     (('S2', 'vid_2'), 0.9771318857464869)],
    'rank': 5,
    'average_precision': 0.24090909090909088},
   'vid_1': {'ranks': [(('S3', 'vid_1'), 0.9939760064728501

In [76]:
results_00_cache_bovw_align = cached_approach(
    vid_ds, M00, mdl_00_vid_threshold, simclr_frame_sim,
    mdl_00_frame_threshold, codebook_00, vw, 19, fps = 30, ftk = 1,
    align_sim = True
)

In [101]:
results_00_cache_bovw_align = cached_approach(
    vid_ds, M00, mdl_00_vid_threshold, simclr_frame_sim,
    0.9, codebook_00, vw, 19, fps = 30, ftk = 1,
    align_sim = True
)

In [102]:
evals_00_bovw_align = evaluate(results_00_bovw_align) # ftk = 1, threshold = 0.9

NameError: name 'results_00_bovw_align' is not defined

In [99]:
evaluate(results_00_cache_bovw_align) # ftk = 1, threshold = 0.0

car_report σ Rank 2.868361901852693
car_report μ Rank 4.85
car_report Median Rank 4.0
car_report mRR: 0.3191040903540904
car_report mAP: 0.26913587909911435
car_report Hit@1: 0.1
car_report Hit@5: 0.65
car_report Hit@10: 0.95
king σ Rank 3.5982634700644147
king μ Rank 5.05
king Median Rank 4.0
king mRR: 0.40775793650793657
king mAP: 0.3214280115595905
king Hit@1: 0.25
king Hit@5: 0.6
king Hit@10: 0.9
tasty σ Rank 1.7578395831246945
tasty μ Rank 2.9
tasty Median Rank 2.0
tasty mRR: 0.5449999999999999
tasty mAP: 0.4172521922521922
tasty Hit@1: 0.35
tasty Hit@5: 1.0
tasty Hit@10: 1.0


{'car_report': {'S0': {'vid_0': {'ranks': [(('S2', 'vid_0'),
      0.808728590372351),
     (('S1', 'vid_0'), 0.8057658754926667),
     (('S4', 'vid_0'), 0.7476912972475713),
     (('S4', 'vid_3'), 0.7464049639609809),
     (('S3', 'vid_0'), 0.6844920842582813),
     (('S3', 'vid_1'), 0.6829318711509527),
     (('S0', 'vid_2'), 0.6631147854954358),
     (('S4', 'vid_2'), 0.4972982385246937),
     (('S3', 'vid_2'), 0.4965941579057019),
     (('S0', 'vid_3'), 0.49582038098202913),
     (('S3', 'vid_3'), 0.49581408071873145),
     (('S0', 'vid_1'), 0.4952371312203373),
     (('S4', 'vid_1'), 0.4948146285343488),
     (('S1', 'vid_2'), 0.49349272585588033),
     (('S1', 'vid_3'), 0.4930592236707055),
     (('S2', 'vid_1'), 0.4925275423108218),
     (('S2', 'vid_3'), 0.4916492342507575),
     (('S1', 'vid_1'), 0.4896655467652016),
     (('S2', 'vid_2'), 0.48856594287324345)],
    'rank': 7,
    'average_precision': 0.1976190476190476},
   'vid_1': {'ranks': [(('S1', 'vid_3'), 0.898623735709

In [86]:
get_eval_results(evals_00_bovw_align, 'tasty', 'average_precision')

0.21666666666666667
0.16402116402116404
0.6333333333333333
0.29870129870129875
0.2853535353535353
0.6388888888888888
0.6388888888888888
0.7555555555555555
0.2888888888888889
0.2686868686868687
0.425
0.3833333333333333
0.4444444444444444
0.35555555555555557
0.12103174603174603
0.27777777777777773
1.0
1.0
1.0
0.9166666666666666


TypeError: 'numpy.float64' object is not iterable

In [111]:
import pickle
results_path = Path('/tf/data/results/M00')

with open(results_path/'results_00.pkl', 'wb') as f:
    pickle.dump(results_00, f, protocol=pickle.HIGHEST_PROTOCOL)

with open(results_path/'evals_00.pkl', 'wb') as f:
    pickle.dump(evals_00, f, protocol=pickle.HIGHEST_PROTOCOL)

with open(results_path/'results_00_bovw_align.pkl', 'wb') as f:
    pickle.dump(results_00_bovw_align, f, protocol=pickle.HIGHEST_PROTOCOL)

with open(results_path/'evals_00_bovw_align.pkl', 'wb') as f:
    pickle.dump(evals_00_bovw_align, f, protocol=pickle.HIGHEST_PROTOCOL)

In [91]:
results_path = Path('/tf/data/results/M00')

eval_sift_bovw_align = pickle.load( open( results_path/'evals_00_bovw_align.pkl', "rb" ) )

In [95]:
eval_sift_bovw_align['car_report']

{'S0': {'vid_0': {'ranks': [(('S0', 'vid_2'), 0.9841517610647245),
    (('S0', 'vid_1'), 0.8032278577774794),
    (('S4', 'vid_0'), 0.7394318735477277),
    (('S0', 'vid_3'), 0.7377530739649625),
    (('S4', 'vid_2'), 0.7364081734894096),
    (('S1', 'vid_2'), 0.7347430196774605),
    (('S2', 'vid_0'), 0.7344045295622541),
    (('S1', 'vid_3'), 0.732794667168452),
    (('S4', 'vid_3'), 0.731848462172171),
    (('S1', 'vid_0'), 0.7287487702039421),
    (('S2', 'vid_3'), 0.7224495133817408),
    (('S3', 'vid_2'), 0.6828160863506159),
    (('S3', 'vid_0'), 0.6828127740633692),
    (('S3', 'vid_3'), 0.6816461100753022),
    (('S3', 'vid_1'), 0.6812751300108997),
    (('S4', 'vid_1'), 0.6808115291281194),
    (('S2', 'vid_1'), 0.6781688814778082),
    (('S1', 'vid_1'), 0.6758977344828558),
    (('S2', 'vid_2'), 0.6731585643976764)],
   'rank': 1,
   'average_precision': 0.9166666666666666},
  'vid_1': {'ranks': [(('S0', 'vid_2'), 0.895140096952731),
    (('S0', 'vid_0'), 0.8032278577774794)

In [100]:
# import json
# class NpEncoder(json.JSONEncoder):
#     def default(self, obj):
#         if isinstance(obj, np.integer):
#             return int(obj)
#         elif isinstance(obj, np.floating):
#             return float(obj)
#         elif isinstance(obj, np.ndarray):
#             return obj.tolist()
#         elif isinstance(obj, tuple):
#             return str(obj)
#         else:
#             return super(NpEncoder, self).default(obj)
# results_path = Path('/tf/data/results/M00')
# with open(results_path/'results_00.json', 'w') as fp:
#     json.dump(results_00, fp, cls = NpEncoder)

# with open(results_path/'evals_00.json', 'w') as fp:
#     json.dump(evals_00, fp, cls = NpEncoder)

# # with open(results_path/'results_00_bovw_align.json', 'w') as fp:
# #     json.dump(results_00_bovw_align, fp)

# with open(results_path/'evals_00_bovw_align.json', 'w') as fp:
#     json.dump(evals_00_bovw_align, fp, cls = NpEncoder)

TypeError: key ('S4', 'vid_0') is not a string

In [31]:
mdl_01_vid_threshold = 0.0
mdl_01_frame_threshold = 0.0
model = SimCLRModel.load_from_checkpoint(checkpoint_path='/tf/data/models/simclr/checkpointepoch=98.ckpt').eval()
M01 = SimCLRExtractor(model)

fname = f'/tf/data/models/cookbook_M01_1000n_100vw.model'
codebook_01 = pickle.load(open(fname, 'rb'))

In [48]:
results_01_cache = cached_approach(
    vid_ds, M01, mdl_01_vid_threshold, simclr_frame_sim,
    mdl_01_frame_threshold, codebook_01, vw, 19, fps = 30, ftk = 1
)

In [66]:
evaluate(results_01_cache) # ftk = 1, no threshold

car_report σ Rank 3.6318039594669758
car_report μ Rank 2.9
car_report Median Rank 1.0
car_report mRR: 0.7559294871794873
car_report mAP: 0.6176476198380222
car_report Hit@5: 0.85
king σ Rank 0.6538348415311012
king μ Rank 1.15
king Median Rank 1.0
king mRR: 0.9625
king mAP: 0.8069936058093953
king Hit@5: 1.0
tasty σ Rank 0.3
tasty μ Rank 1.1
tasty Median Rank 1.0
tasty mRR: 0.95
tasty mAP: 0.8740277777777777
tasty Hit@5: 1.0


{'car_report': {'S0': {'vid_0': {'ranks': [(('S4', 'vid_3'),
      0.9179039973367417),
     (('S1', 'vid_2'), 0.916244867784407),
     (('S2', 'vid_0'), 0.8988486868327271),
     (('S1', 'vid_0'), 0.8926742868310932),
     (('S2', 'vid_1'), 0.886525676974221),
     (('S4', 'vid_0'), 0.886018344735942),
     (('S4', 'vid_2'), 0.886018344735942),
     (('S2', 'vid_3'), 0.8846630799210273),
     (('S4', 'vid_1'), 0.8826717449140168),
     (('S2', 'vid_2'), 0.8704217558863261),
     (('S1', 'vid_1'), 0.8674557230011675),
     (('S0', 'vid_2'), 0.8669034323511741),
     (('S1', 'vid_3'), 0.8322814715939507),
     (('S0', 'vid_1'), 0.8062496404103776),
     (('S3', 'vid_3'), 0.7864373613335622),
     (('S3', 'vid_0'), 0.7545381883890863),
     (('S3', 'vid_1'), 0.7221121934825507),
     (('S3', 'vid_2'), 0.650566822070925),
     (('S0', 'vid_3'), 0.6222576366794502)],
    'rank': 12,
    'average_precision': 0.1280284043441938},
   'vid_1': {'ranks': [(('S2', 'vid_1'), 0.9127318198237152),


In [49]:
evaluate(results_01_cache) # ftk = 1, no threshold

car_report σ Rank 3.6318039594669758
car_report μ Rank 2.9
car_report Median Rank 1.0
car_report mRR: 0.7559294871794873
car_report mAP: 0.6176476198380222
king σ Rank 0.6538348415311012
king μ Rank 1.15
king Median Rank 1.0
king mRR: 0.9625
king mAP: 0.8069936058093953
tasty σ Rank 0.3
tasty μ Rank 1.1
tasty Median Rank 1.0
tasty mRR: 0.95
tasty mAP: 0.8740277777777777


{'car_report': {'S0': {'vid_0': {'ranks': [(('S4', 'vid_3'),
      0.9179039973367417),
     (('S1', 'vid_2'), 0.916244867784407),
     (('S2', 'vid_0'), 0.8988486868327271),
     (('S1', 'vid_0'), 0.8926742868310932),
     (('S2', 'vid_1'), 0.886525676974221),
     (('S4', 'vid_0'), 0.886018344735942),
     (('S4', 'vid_2'), 0.886018344735942),
     (('S2', 'vid_3'), 0.8846630799210273),
     (('S4', 'vid_1'), 0.8826717449140168),
     (('S2', 'vid_2'), 0.8704217558863261),
     (('S1', 'vid_1'), 0.8674557230011675),
     (('S0', 'vid_2'), 0.8669034323511741),
     (('S1', 'vid_3'), 0.8322814715939507),
     (('S0', 'vid_1'), 0.8062496404103776),
     (('S3', 'vid_3'), 0.7864373613335622),
     (('S3', 'vid_0'), 0.7545381883890863),
     (('S3', 'vid_1'), 0.7221121934825507),
     (('S3', 'vid_2'), 0.650566822070925),
     (('S0', 'vid_3'), 0.6222576366794502)],
    'rank': 12,
    'average_precision': 0.1280284043441938},
   'vid_1': {'ranks': [(('S2', 'vid_1'), 0.9127318198237152),


In [61]:
evaluate(results_01_bovw_align_cache) # ftk = 1, no threshold

car_report σ Rank 2.2555487137279924
car_report μ Rank 2.25
car_report Median Rank 1.0
car_report mRR: 0.75
car_report mAP: 0.6847485567551357
king σ Rank 0.6403124237432849
king μ Rank 1.3
king Median Rank 1.0
king mRR: 0.8833333333333332
king mAP: 0.7433590921748816
tasty σ Rank 0.47696960070847283
tasty μ Rank 1.15
tasty Median Rank 1.0
tasty mRR: 0.9416666666666667
tasty mAP: 0.8956150793650794


{'car_report': {'S0': {'vid_0': {'ranks': [(('S4', 'vid_3'),
      0.9255708677081183),
     (('S4', 'vid_0'), 0.9163295903019648),
     (('S4', 'vid_2'), 0.9046270163402831),
     (('S2', 'vid_0'), 0.9026971596240374),
     (('S0', 'vid_2'), 0.9024383721601931),
     (('S1', 'vid_0'), 0.9011721377581385),
     (('S1', 'vid_2'), 0.8960606406178538),
     (('S2', 'vid_1'), 0.8905258971760159),
     (('S4', 'vid_1'), 0.8861024866292022),
     (('S1', 'vid_1'), 0.8853016075725052),
     (('S2', 'vid_3'), 0.8834868577173784),
     (('S0', 'vid_1'), 0.8740850928882264),
     (('S2', 'vid_2'), 0.8740055669891751),
     (('S3', 'vid_3'), 0.8619039831362466),
     (('S1', 'vid_3'), 0.8559371802395124),
     (('S3', 'vid_0'), 0.8317610108245463),
     (('S3', 'vid_1'), 0.8125996927970479),
     (('S3', 'vid_2'), 0.773305005056518),
     (('S0', 'vid_3'), 0.7632954981723956)],
    'rank': 5,
    'average_precision': 0.17485380116959068},
   'vid_1': {'ranks': [(('S2', 'vid_1'), 0.907841045893894

In [60]:
results_01_bovw_align_cache = cached_approach(
    vid_ds, M01, mdl_01_vid_threshold, simclr_frame_sim,
    mdl_01_frame_threshold, codebook_01, vw, 19, fps = 30, ftk = 1,
    align_sim = True
)

In [63]:
results_01_bovw_align_cache = cached_approach(
    vid_ds, M01, mdl_01_vid_threshold, simclr_frame_sim,
    0.9, codebook_01, vw, 19, fps = 30, ftk = 1,
    align_sim = True
)

In [65]:
evaluate(results_01_bovw_align_cache) # ftk = 1, threshold = 0.9

car_report σ Rank 1.9358460682605938
car_report μ Rank 2.55
car_report Median Rank 2.0
car_report mRR: 0.6245833333333334
car_report mAP: 0.5270158033973823
car_report Hit@5: 0.9
king σ Rank 1.5644487847162016
king μ Rank 1.55
king Median Rank 1.0
king mRR: 0.8729166666666666
king mAP: 0.781579439079439
king Hit@5: 0.95
tasty σ Rank 0.3570714214271425
tasty μ Rank 1.15
tasty Median Rank 1.0
tasty mRR: 0.925
tasty mAP: 0.9129761904761905
tasty Hit@5: 1.0


{'car_report': {'S0': {'vid_0': {'ranks': [(('S0', 'vid_2'),
      0.7105213571866889),
     (('S4', 'vid_3'), 0.6802873402013959),
     (('S0', 'vid_1'), 0.67666704225469),
     (('S2', 'vid_0'), 0.6708428996562334),
     (('S1', 'vid_0'), 0.6681127248548827),
     (('S4', 'vid_0'), 0.6646736573721841),
     (('S4', 'vid_2'), 0.6646495307365267),
     (('S2', 'vid_1'), 0.6642186772468507),
     (('S2', 'vid_3'), 0.6637950910844507),
     (('S1', 'vid_2'), 0.663617529807468),
     (('S4', 'vid_1'), 0.6620987842270872),
     (('S1', 'vid_1'), 0.6550957173673339),
     (('S1', 'vid_3'), 0.6339922653157386),
     (('S3', 'vid_3'), 0.6091770242786043),
     (('S2', 'vid_2'), 0.6014883792613529),
     (('S3', 'vid_0'), 0.5984580235404158),
     (('S3', 'vid_1'), 0.5818367243363292),
     (('S3', 'vid_2'), 0.5464383524883365),
     (('S0', 'vid_3'), 0.47716250354709205)],
    'rank': 1,
    'average_precision': 0.6081871345029239},
   'vid_1': {'ranks': [(('S4', 'vid_0'), 0.6899753222571436)

In [64]:
evaluate(results_01_bovw_align_cache) # ftk = 1, threshold = 0.9

car_report σ Rank 1.9358460682605938
car_report μ Rank 2.55
car_report Median Rank 2.0
car_report mRR: 0.6245833333333334
car_report mAP: 0.5270158033973823
king σ Rank 1.5644487847162016
king μ Rank 1.55
king Median Rank 1.0
king mRR: 0.8729166666666666
king mAP: 0.781579439079439
tasty σ Rank 0.3570714214271425
tasty μ Rank 1.15
tasty Median Rank 1.0
tasty mRR: 0.925
tasty mAP: 0.9129761904761905


{'car_report': {'S0': {'vid_0': {'ranks': [(('S0', 'vid_2'),
      0.7105213571866889),
     (('S4', 'vid_3'), 0.6802873402013959),
     (('S0', 'vid_1'), 0.67666704225469),
     (('S2', 'vid_0'), 0.6708428996562334),
     (('S1', 'vid_0'), 0.6681127248548827),
     (('S4', 'vid_0'), 0.6646736573721841),
     (('S4', 'vid_2'), 0.6646495307365267),
     (('S2', 'vid_1'), 0.6642186772468507),
     (('S2', 'vid_3'), 0.6637950910844507),
     (('S1', 'vid_2'), 0.663617529807468),
     (('S4', 'vid_1'), 0.6620987842270872),
     (('S1', 'vid_1'), 0.6550957173673339),
     (('S1', 'vid_3'), 0.6339922653157386),
     (('S3', 'vid_3'), 0.6091770242786043),
     (('S2', 'vid_2'), 0.6014883792613529),
     (('S3', 'vid_0'), 0.5984580235404158),
     (('S3', 'vid_1'), 0.5818367243363292),
     (('S3', 'vid_2'), 0.5464383524883365),
     (('S0', 'vid_3'), 0.47716250354709205)],
    'rank': 1,
    'average_precision': 0.6081871345029239},
   'vid_1': {'ranks': [(('S4', 'vid_0'), 0.6899753222571436)

In [75]:
evals_01_bovw_align = evaluate(results_01_bovw_align) # ftk = 1, threshold = 0.9

car_report σ Rank 3.528101472463625
car_report μ Rank 3.05
car_report Median Rank 2.0
car_report mRR: 0.6248511904761905
car_report mAP: 0.5223357816217723
king σ Rank 1.5652475842498528
king μ Rank 1.5
king Median Rank 1.0
king mRR: 0.8979166666666666
king mAP: 0.7741214779372674
tasty σ Rank 0.3
tasty μ Rank 1.1
tasty Median Rank 1.0
tasty mRR: 0.95
tasty mAP: 0.9030555555555555


In [28]:
results_01 = approach(
    vid_ds, M01, mdl_01_vid_threshold, simclr_frame_sim,
    mdl_01_frame_threshold, codebook_01, vw, 19, fps = 30, ftk = 1
)

car_report
king
tasty



In [ ]:
results_01_bovw_align = approach(
    vid_ds, M01, mdl_01_vid_threshold, sift_frame_sim,
    mdl_01_frame_threshold, codebook_01, vw, 19, fps = 30, ftk = 1,
    align_sim = True
)

car_report
king
tasty



Fuzzy: 3.0 8
Fuzzy: 3.0 6
Fuzzy: 2.0 8
Fuzzy: 3.0 8
Fuzzy: 3.0 8
Fuzzy: 3.0 8
Fuzzy: 3.0 8
Fuzzy: 3.0 8
Fuzzy: 4.0 8
Fuzzy: 3.0 8
Fuzzy: 3.0 8
Fuzzy: 3.0 8
Fuzzy: 3.0 8
Fuzzy: 3.0 8
Fuzzy: 3.0 8
Fuzzy: 3.0 8
Fuzzy: 3.0 8
Fuzzy: 3.0 8


In [40]:
evals_01_bovw = evaluate(results_01) # ftk = 1

car_report σ Rank 3.6318039594669758
car_report μ Rank 2.9
car_report Median Rank 1.0
car_report mRR: 0.7559294871794873
car_report mAP: 0.6176476198380222
king σ Rank 0.6538348415311012
king μ Rank 1.15
king Median Rank 1.0
king mRR: 0.9625
king mAP: 0.8069936058093953
tasty σ Rank 0.3
tasty μ Rank 1.1
tasty Median Rank 1.0
tasty mRR: 0.95
tasty mAP: 0.8740277777777777


In [87]:
get_eval_results(evals_01, 'car_report', 'average_precision')

0.4666666666666666
0.7222222222222222
0.8055555555555555
1.0
1.0
1.0
1.0
1.0
1.0
0.9166666666666666
1.0
1.0
0.40277777777777773
0.9166666666666666
0.8666666666666667
1.0
0.5777777777777777
1.0
1.0
0.8055555555555555


TypeError: 'numpy.float64' object is not iterable

In [91]:
get_eval_results(evals_01_bovw_align, 'car_report', 'rank')

2
16
1
1
3
3
3
2
7
1
2
8
1
1
1
1
3
1
3
1


TypeError: 'numpy.float64' object is not iterable

In [112]:
import pickle
results_path = Path('/tf/data/results/M01')

with open(results_path/'results_01.pkl', 'wb') as f:
    pickle.dump(results_01, f, protocol=pickle.HIGHEST_PROTOCOL)

with open(results_path/'evals_01.pkl', 'wb') as f:
    pickle.dump(evals_01, f, protocol=pickle.HIGHEST_PROTOCOL)

with open(results_path/'results_01_bovw_align.pkl', 'wb') as f:
    pickle.dump(results_01_bovw_align, f, protocol=pickle.HIGHEST_PROTOCOL)

with open(results_path/'evals_01_bovw_align.pkl', 'wb') as f:
    pickle.dump(evals_01_bovw_align, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
vw = 100

M00 = SIFTExtractor(cv2.xfeatures2d.SIFT_create())
fname = f'/tf/data/models/cookbook_M00_1000n_100vw.model'
codebook_00 = pickle.load(open(fname, 'rb'))

In [ ]:
m = len(vid_0)
n = len(vid_1)

fuzzy_LCS(vid_0, vid_1, 10, 10, sift_frame_sim, M00, codebook_00, vw, 0.955, fps = 30, frames_to_keep = 30)

In [ ]:
model = SimCLRModel.load_from_checkpoint(checkpoint_path='/tf/data/models/simclr/checkpointepoch=98.ckpt').eval()
M01 = SimCLRExtractor(model)

fname = f'/tf/data/models/cookbook_M01_1000n_100vw.model'
codebook_01 = pickle.load(open(fname, 'rb'))

In [ ]:
m = len(vid_0)
n = len(vid_1)

fuzzy_LCS(vid_0, vid_1, 10, 10, simclr_frame_sim, M01, codebook_01, vw, 0.76, frames_to_keep = 30)

In [ ]:
# From geeksforgeeks: https://www.geeksforgeeks.org/longest-common-substring-dp-29/
def fuzzy_LCS(X, Y, m, n):
    # Create a table to store lengths of 
    # longest common suffixes of substrings.  
    # Note that LCSuff[i][j] contains the  
    # length of longest common suffix of  
    # X[0...i-1] and Y[0...j-1]. The first 
    # row and first column entries have no 
    # logical meaning, they are used only 
    # for simplicity of the program. 
      
    # LCSuff is the table with zero  
    # value initially in each cell 
    LCSuff = [[0 for k in range(n+1)] for l in range(m+1)] 
      
    # To store the length of  
    # longest common substring 
    result = 0 
  
    # Following steps to build 
    # LCSuff[m+1][n+1] in bottom up fashion 
    for i in range(m + 1): 
        for j in range(n + 1): 
            if (i == 0 or j == 0): 
                LCSuff[i][j] = 0
            elif (X[i-1] == Y[j-1]): 
                LCSuff[i][j] = LCSuff[i-1][j-1] + 1
                result = max(result, LCSuff[i][j]) 
            else: 
                LCSuff[i][j] = 0
    return result 

In [ ]:
X = 'aaaaaaaaaaaaaa'
Y = 'aaaaaaaaaaaaaa'
  
m = len(X) 
n = len(Y) 
  
print('Length of Longest Common Substring is', 
                      fuzzy_LCS(X, Y, m, n)) 

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()